In [25]:
from google.cloud import bigquery
import pandas as pd

In [26]:
client = bigquery.Client()

query = """
SELECT *
FROM `diabetes-readmission-data.diabetes_readmission_dataset.diabetic_data`
"""

df = client.query(query).to_dataframe()

In [27]:
df.shape

(101766, 50)

In [28]:
df.dtypes

encounter_id                object
patient_nbr                 object
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            Int64
discharge_disposition_id     Int64
admission_source_id          Int64
time_in_hospital             Int64
payer_code                  object
medical_specialty           object
num_lab_procedures           Int64
num_procedures               Int64
num_medications              Int64
number_outpatient            Int64
number_emergency             Int64
number_inpatient             Int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             Int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [29]:
# Check for missing values represented as '?'
for col in df.columns:
    if df[col].dtype == 'object':
        print(col, df[col][df[col] == '?'].count())

encounter_id 0
patient_nbr 0
race 2273
gender 0
age 0
weight 98569
payer_code 40256
medical_specialty 49949
diag_1 21
diag_2 358
diag_3 1423
max_glu_serum 0
A1Cresult 0
metformin 0
repaglinide 0
nateglinide 0
chlorpropamide 0
glimepiride 0
acetohexamide 0
glipizide 0
glyburide 0
tolbutamide 0
pioglitazone 0
rosiglitazone 0
acarbose 0
miglitol 0
troglitazone 0
tolazamide 0
examide 0
citoglipton 0
insulin 0
glyburide_metformin 0
glipizide_metformin 0
glimepiride_pioglitazone 0
metformin_rosiglitazone 0
metformin_pioglitazone 0
change 0
diabetesMed 0
readmitted 0


In [30]:
# Check for 'Unknown/Invalid' values in gender column
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count())

gender 3
